In [44]:
print("=== PROCEDURE, REFERENCE, & LIMITATIONS ===")
print("Procedure: Direct Design Method")
print("Reference: ACI 318M-14 | RCD by Darwin & Nelson")
print("Limitations: \n minimum 3x3 panel. If fewer, interior negative moments tend to be too small. \n Ratio not greater than 2. \n Span lengths not differ by more than 1/3 of the longer span. \n applicable only to braced frames \n Service live load shall not exceed two times the service dead load. \n relative stiffness of beams in two perpedicular directions shall not be lt 0.2 or gt 5.0.")

=== PROCEDURE, REFERENCE, & LIMITATIONS ===
Procedure: Direct Design Method
Reference: ACI 318M-14 | RCD by Darwin & Nelson
Limitations: 
 minimum 3x3 panel. If fewer, interior negative moments tend to be too small. 
 Ratio not greater than 2. 
 Span lengths not differ by more than 1/3 of the longer span. 
 applicable only to braced frames 
 Service live load shall not exceed two times the service dead load. 
 relative stiffness of beams in two perpedicular directions shall not be lt 0.2 or gt 5.0.


In [45]:
import math
pi = math.pi
import pandas as pd
def sqrt(x):
    return math.sqrt(x)
def rd(x):
    return round(x,2)
def roundup(x):
    return math.ceil(x/10.0) * 10

In [46]:
#conversions
def mm_to_in(x):
    return round(x/25.4,2)
def kPa_to_psf(x):
    return round(x*144/6.89655172413793,2)
def psf_to_kPa(x):
    return round(x*6.89655172413793/144,2)
def kNm_to_kipft(x):
    """
    Output:
    Converts kN-m to kip-ft
    """
    return rd(x/1.35582)
def kipft_to_kNm(x):
    """
    Output:
    Converts kip-ft to kN-m
    """
    return rd(x*1.35582)

In [47]:
#Class and methods
class Beam:
    def __init__ (self, name, dimy, dimz, length):
        self.name = name
        self.dimy = dimy
        self.dimz = dimz
        self.length  = length
    def Icb(self):
        return self.dimy*self.dimz**3/12
class Slab:
    def __init__ (self, name, L, W, thk):
        self.name = name
        self.L = L #The larger dimension
        self.W = W 
        self.thk = thk
    def Ics(self):
        return self.W*self.thk**3/12

In [48]:
#Slab and Beam Material Properties
fc = 27 #MPa
Ec = rd(4700*sqrt(fc)/1000) #GPa, for normal weight concrete
fy = 230 #MPa
print("=== MATERIAL PROPERTIES ===")
print(f"fc = {fc}MPa | fy = {fy}MPa | Ec = {Ec}GPa")

#Beam Material and Geometric Properties
B_north = Beam("B_north",300,600,7700)
B_south = Beam("B_south",300,600,7700)
B_east = Beam("B_east",300,600,7700)
B_west = Beam("B_west",300,600,7700)

#Slab Material and Geometric Properties
print("=== GEOMETRIC PROPERTIES ===")
S1 = Slab("S1",7260,5730,150)
lb = S1.L #larger dimension
la = S1.W #shorter dimension
print(f"Initial Slab Geometric Properties\n L = {S1.L}mm, W = {S1.W}mm, thk = {S1.thk}mm.")
beta = S1.L / S1.W #Ratio of longer to shorter slab dim
print(f"Ratio of longer to shorter slab clear span: {rd(beta)}")

#Beam-to-slab Stiffness Ratio, alpha_f
alpha_f = B_north.Icb() / S1.Ics()
print(f"Average Relative Stiffness beam to slab, alpha_f = {rd(alpha_f)}.")


=== MATERIAL PROPERTIES ===
fc = 27MPa | fy = 230MPa | Ec = 24.42GPa
=== GEOMETRIC PROPERTIES ===
Initial Slab Geometric Properties
 L = 7260mm, W = 5730mm, thk = 150mm.
Ratio of longer to shorter slab clear span: 1.27
Average Relative Stiffness beam to slab, alpha_f = 3.35.


In [49]:
#Slab Minimum Thickness per 8.3.1.2 ACI318M-14
print("=== MIN. THICKNESS (8.3.1.2) ===")
if alpha_f > 0.20 and alpha_f < 2:
    print(f"1. case 1 governs: 0.20 < alpha_f < 2.0")
    h_min = rd(max( S1.L * (0.8 + fy/1400) / (36+5*(S1.L/S1.W)*(alpha_f - 0.2)) , 125)) #mm
else: 
    print(f"1. case 2 governs: alpha_f > 2.0")
    h_min = rd(max( S1.L * (0.8 + fy/1400) / (36+9*(S1.L/S1.W)) , 90)) #mm
    
print(f"2. Minimum Slab Thickness (ACI318M-14, 8.3.1.2) = {h_min}mm.")
S1.thk = roundup(max(S1.thk,h_min))
print(f"3. Adopt S1.thk = {S1.thk}mm ({mm_to_in(S1.thk)}in) ")
#Factored Uniform Load
print("=== APPLIED LOADS ===")
DL = rd(24*max(h_min,S1.thk)*.001 + 0.2) #selfweight of slab, kPa
LL = 7.0 #kPa
qa = DL + LL
qu = 1.2*DL + 1.6*LL
print(f"1. DL = {DL}kPa ({kPa_to_psf(DL)}psf) | LL = {LL}kPa ({kPa_to_psf(LL)}psf)")
print(f"2. Service Uniform Pressure, qa = {rd(qa)}kPa ({kPa_to_psf(qa)}psf)")
print(f"3. Factored Uniform Pressure, qu = {rd(qu)}kPa ({kPa_to_psf(qu)}psf)") #kPa

=== MIN. THICKNESS (8.3.1.2) ===
1. case 2 governs: alpha_f > 2.0
2. Minimum Slab Thickness (ACI318M-14, 8.3.1.2) = 147.68mm.
3. Adopt S1.thk = 150mm (5.91in) 
=== APPLIED LOADS ===
1. DL = 3.8kPa (79.34psf) | LL = 7.0kPa (146.16psf)
2. Service Uniform Pressure, qa = 10.8kPa (225.5psf)
3. Factored Uniform Pressure, qu = 15.76kPa (329.07psf)


In [50]:
#Minimum reinforcement near tension face for each dir
print("=== MIN. REINF. NEAR TENSION FACE, EA.DIR. (8.6.1) ===")
if fy < 420:
    Asmin = rd(0.0020 * 1000 * S1.thk)
    print(f"1. Asmin = 0.0020Ag = {Asmin}mm2 per mtr width of slab")
else:
    Asmin = rd( max( 0.0018*420*1000*S1.thk/fy , 0.0014*1000*S1.thk ) )
    print(f"1. Asmin = {Asmin}mm2 per mtr width of slab")

#Minimum spacing of flexural reinforcement at crit sections
print("1. Max. spacing of flex. reinf. at crit. sec. must not exceed 2*slab_thk per 8.7.3")
s_max = int(2*S1.thk) #mm
print(f"2. s_max = {s_max}mm")

#Corner restraint in slab
print("=== CORNER RESTRAINT IN SLAB (8.7.3) ===")
lcor = int(rd(lb / 5)) #mm
print(f"1. Length corner bar, lcor = {lcor}mm")
Mu_cor = rd((1/2)*qu*(lcor*0.001)**2) #kN-m
print(f"2. Mu_cor = {Mu_cor}kN-m")


=== MIN. REINF. NEAR TENSION FACE, EA.DIR. (8.6.1) ===
1. Asmin = 0.0020Ag = 300.0mm2 per mtr width of slab
1. Max. spacing of flex. reinf. at crit. sec. must not exceed 2*slab_thk per 8.7.3
2. s_max = 300mm
=== CORNER RESTRAINT IN SLAB (8.7.3) ===
1. Length corner bar, lcor = 1452mm
2. Mu_cor = 16.61kN-m


In [51]:
#Case: Simply Supported Square Slab, if only bending moments were present
print("=== CASE: SIMPLY SUPPORTED SQUARE SLAB ===")
print("1. Twisting moment relieves 25 percent of bending moment.")
print("2. Bending moment formula: (qu/2)*l**2 / 8 less 25%")
Mu_square = ( ((qu*0.5)*(lb*0.001)**2) / 8 ) * 0.75 #kN-m
print(f"3. Mu_squareslab = {rd(Mu_square)}kN-m per mtr width")

=== CASE: SIMPLY SUPPORTED SQUARE SLAB ===
1. Twisting moment relieves 25 percent of bending moment.
2. Bending moment formula: (qu/2)*l**2 / 8 less 25%
3. Mu_squareslab = 38.94kN-m per mtr width


In [52]:
#Total Statical Moment, Mo (CONSIDER BENDING OF THE LONGER SPAN)
print("=== TOTAL STATICAL MOMENT, Mo ===")
Mo_LDir = qu*S1.W*.001*(S1.L*.001)**2 / 8 #kN-m
print(f"1a. Total Statical Moment, Mo_LDir = {rd(Mo_LDir)}kN-m ({kNm_to_kipft(Mo_LDir)}kip-ft) long dir.")
Mo_SDir = qu*S1.L*.001*(S1.W*.001)**2 / 8 #kN-m
print(f"1b. Total Statical Moment, Mo_SDir = {rd(Mo_SDir)}kN-m ({kNm_to_kipft(Mo_SDir)}kip-ft) shrt dir.")
#Widths of Middle and Column Strip...
width_cs = rd(min(0.5*S1.L,0.5*S1.W)) # 8.4.1.5
print(f"2a. Width of Column Strip: {width_cs}mm ({mm_to_in(width_cs/12)}ft)") 
width_ms = rd(max(S1.L - width_cs, S1.W - width_cs))
print(f"2b. Width of Middle Strip: {width_ms}mm ({mm_to_in(width_ms/12)}ft)")

=== TOTAL STATICAL MOMENT, Mo ===
1a. Total Statical Moment, Mo_LDir = 594.97kN-m (438.83kip-ft) long dir.
1b. Total Statical Moment, Mo_SDir = 469.58kN-m (346.35kip-ft) shrt dir.
2a. Width of Column Strip: 2865.0mm (9.4ft)
2b. Width of Middle Strip: 4395.0mm (14.42ft)


In [53]:
#Interior Panel Moments Distribution of Mo
print("=== LDIR: Distribuition of Mo for Interior Panels (8.10.4.1) ===")
Mol_neg = rd(0.65*Mo_LDir) #at supports/ends of slab
Mol_pos = rd(0.35*Mo_LDir) #at midspan of slab
print(f" 1a. Interior Panel Moment (negative), Mol_neg = {Mol_neg}kNm ({kNm_to_kipft(Mol_neg)}kipft)")
print(f" 1b. Interior Panel Moment (positive), Mol_pos = {Mol_pos}kNm ({kNm_to_kipft(Mol_pos)}kipft)")
#Interior Panel, Column Strip, Table 8.10.5.1
Mol_neg_cs = rd(0.75*Mol_neg)
print(f"  2a. IPM_neg, Column Strip, Mol_neg_cs = {Mol_neg_cs}kNm ({kNm_to_kipft(Mol_neg_cs)}kipft)")
Mol_neg_cs_beam = rd(0.85*Mol_neg_cs)
Mol_neg_cs_slab = rd(0.15*Mol_neg_cs)
print(f"    2a.1. Mol_neg_cs_beam (85%, 8.10.5.7): {Mol_neg_cs_beam}kNm ({kNm_to_kipft(Mol_neg_cs_beam)}kipft)")
print(f"    2a.1. Mol_neg_cs_slab (15%, 8.10.5.7): {Mol_neg_cs_slab}kNm ({kNm_to_kipft(Mol_neg_cs_slab)}kipft)")
#Interior Panel, Middle Strip
Mol_neg_ms = rd(Mol_neg - Mol_neg_cs) #8.10.6
print(f"  2b. IPM_neg, Middle Strip, Mol_neg_ms = {Mol_neg_ms}kNm ({kNm_to_kipft(Mol_neg_ms)}kipft)")
#Pos Mu, Column Strip, Table 8.10.5.5
Mol_pos_cs = rd(0.75*Mol_pos)
print(f"  2c. IPM_pos, Column Strip, Mol_pos_cs = {Mol_pos_cs}kNm ({kNm_to_kipft(Mol_pos_cs)}kipft)")
Mol_pos_cs_beam = rd(0.85*Mol_pos_cs)
Mol_pos_cs_slab = rd(0.15*Mol_pos_cs)
print(f"    2c.1. Mol_pos_cs_beam (85%, 8.10.5.7): {Mol_pos_cs_beam}kNm ({kNm_to_kipft(Mol_pos_cs_beam)}kipft)")
print(f"    2c.1. Mol_pos_cs_slab (15%, 8.10.5.7): {Mol_pos_cs_slab}kNm ({kNm_to_kipft(Mol_pos_cs_slab)}kipft)")
Mol_pos_ms = rd(Mol_pos - Mol_pos_cs) #8.10.6
print(f"  2d. IPM_pos, Mid Strip, Mol_pos_ms = {Mol_pos_ms}kNm ({kNm_to_kipft(Mol_pos_ms)}kipft)")

print("=== SDIR: Distribuition of Mo for Interior Panels (8.10.4.1) ===")
Mos_neg = rd(0.65*Mo_SDir) #at supports/ends of slab
Mos_pos = rd(0.35*Mo_SDir) #at midspan of slab
print(f" 1a. Interior Panel Moment (negative), Mos_neg = {Mos_neg}kNm ({kNm_to_kipft(Mos_neg)}kipft)")
print(f" 1b. Interior Panel Moment (positive), Mos_pos = {Mos_pos}kNm ({kNm_to_kipft(Mos_pos)}kipft)")
#Interior Panel, Column Strip, Table 8.10.5.1
Mos_neg_cs = rd(0.75*Mos_neg)
print(f"  2a. IPM_neg, Column Strip, Mos_neg_cs = {Mos_neg_cs}kNm ({kNm_to_kipft(Mos_neg_cs)}kipft)")
Mos_neg_cs_beam = rd(0.85*Mos_neg_cs)
Mos_neg_cs_slab = rd(0.15*Mos_neg_cs)
print(f"    2a.1. Mos_neg_cs_beam (85%, 8.10.5.7): {Mos_neg_cs_beam}kNm ({kNm_to_kipft(Mos_neg_cs_beam)}kipft)")
print(f"    2a.1. Mos_neg_cs_slab (15%, 8.10.5.7): {Mos_neg_cs_slab}kNm ({kNm_to_kipft(Mos_neg_cs_slab)}kipft)")
#Interior Panel, Middle Strip
Mos_neg_ms = rd(Mos_neg - Mos_neg_cs) #8.10.6
print(f"  2b. IPM_neg, Middle Strip, Mos_neg_ms = {Mos_neg_ms}kNm ({kNm_to_kipft(Mos_neg_ms)}kipft)")
#Pos Mu, Column Strip, Table 8.10.5.5
Mos_pos_cs = rd(0.75*Mos_pos)
print(f"  2c. IPM_pos, Column Strip, Mos_pos_cs = {Mos_pos_cs}kNm ({kNm_to_kipft(Mos_pos_cs)}kipft)")
Mos_pos_cs_beam = rd(0.85*Mos_pos_cs)
Mos_pos_cs_slab = rd(0.15*Mos_pos_cs)
print(f"    2c.1. Mos_pos_cs_beam (85%, 8.10.5.7): {Mos_pos_cs_beam}kNm ({kNm_to_kipft(Mos_pos_cs_beam)}kipft)")
print(f"    2c.1. Mos_pos_cs_slab (15%, 8.10.5.7): {Mos_pos_cs_slab}kNm ({kNm_to_kipft(Mos_pos_cs_slab)}kipft)")
Mos_pos_ms = rd(Mos_pos - Mos_pos_cs) #8.10.6
print(f"  2d. IPM_pos, Mid Strip, Mos_pos_ms = {Mos_pos_ms}kNm ({kNm_to_kipft(Mos_pos_ms)}kipft)")


=== LDIR: Distribuition of Mo for Interior Panels (8.10.4.1) ===
 1a. Interior Panel Moment (negative), Mol_neg = 386.73kNm (285.24kipft)
 1b. Interior Panel Moment (positive), Mol_pos = 208.24kNm (153.59kipft)
  2a. IPM_neg, Column Strip, Mol_neg_cs = 290.05kNm (213.93kipft)
    2a.1. Mol_neg_cs_beam (85%, 8.10.5.7): 246.54kNm (181.84kipft)
    2a.1. Mol_neg_cs_slab (15%, 8.10.5.7): 43.51kNm (32.09kipft)
  2b. IPM_neg, Middle Strip, Mol_neg_ms = 96.68kNm (71.31kipft)
  2c. IPM_pos, Column Strip, Mol_pos_cs = 156.18kNm (115.19kipft)
    2c.1. Mol_pos_cs_beam (85%, 8.10.5.7): 132.75kNm (97.91kipft)
    2c.1. Mol_pos_cs_slab (15%, 8.10.5.7): 23.43kNm (17.28kipft)
  2d. IPM_pos, Mid Strip, Mol_pos_ms = 52.06kNm (38.4kipft)
=== SDIR: Distribuition of Mo for Interior Panels (8.10.4.1) ===
 1a. Interior Panel Moment (negative), Mos_neg = 305.23kNm (225.13kipft)
 1b. Interior Panel Moment (positive), Mos_pos = 164.35kNm (121.22kipft)
  2a. IPM_neg, Column Strip, Mos_neg_cs = 228.92kNm (168.84

In [55]:
#End span Moments Distribution of Mo ACI318M-14 Table 8.10.4.2
print("=== LDIR: Distribution of Mo for End Span with beams between all supports ===")
Mol_eneg = rd(0.16*Mo_LDir)
Mol_ineg = rd(0.70*Mo_LDir)
Mol_poss = rd(0.57*Mo_LDir)
print(f" End Span Panel Moment (negative exterior), Mo_eneg = {Mol_eneg}kNm.")
print(f" End Span Panel Moment (negative interior), Mo_ineg = {Mol_ineg}kNm.")
print(f" End Span Panel Moment (positive), Mo_epos = {Mol_poss}kNm.")
Mol_eneg_cs = rd(0.65*Mol_eneg)
print(f"    1. ESPM_eneg, Col Strip, Mol_eneg_cs = {Mol_eneg_cs}kNm ({kNm_to_kipft(Mol_eneg_cs)}kipft)")
Mol_eneg_cs_beam = rd(0.85*Mol_eneg_cs)
Mol_eneg_cs_slab = rd(0.15*Mol_eneg_cs)
print(f"      1a. Mol_eneg_cs_beam (85%, 8.10.5.7): {Mol_eneg_cs_beam}kNm ({kNm_to_kipft(Mol_eneg_cs_beam)}kipft)")
print(f"      1b. Mol_eneg_cs_slab (15%, 8.10.5.7): {Mol_eneg_cs_slab}kNm ({kNm_to_kipft(Mol_eneg_cs_slab)}kipft)")
Mol_ineg_cs = rd(0.75*Mol_ineg)
print(f"    2. ESPM_ineg, Col Strip, Mol_ineg_cs = {Mol_ineg_cs}kNm ({kNm_to_kipft(Mol_ineg_cs)}kipft)")
Mol_ineg_cs_beam = rd(0.85*Mol_ineg_cs)
Mol_ineg_cs_slab = rd(0.15*Mol_ineg_cs)
print(f"      2a. Mol_ineg_cs_beam (85%, 8.10.5.7): {Mol_ineg_cs_beam}kNm ({kNm_to_kipft(Mol_ineg_cs_beam)}kipft)")
print(f"      2b. Mol_ineg_cs_slab (15%, 8.10.5.7): {Mol_ineg_cs_slab}kNm ({kNm_to_kipft(Mol_ineg_cs_slab)}kipft)")
Mol_poss_cs = rd(0.75*Mol_poss)
print(f"    3. ESPM_pos, Col Strip, Mol_poss_cs = {Mol_poss_cs}kNm ({kNm_to_kipft(Mol_poss_cs)}kipft)")
Mol_poss_cs_beam = rd(0.85*Mol_poss_cs)
Mol_poss_cs_slab = rd(0.15*Mol_poss_cs)
print(f"      3a. Mol_poss_cs_beam (85%, 8.10.5.7): {Mol_poss_cs_beam}kNm ({kNm_to_kipft(Mol_poss_cs_beam)}kipft)")
print(f"      3b. Mol_poss_cs_slab (15%, 8.10.5.7): {Mol_poss_cs_slab}kNm ({kNm_to_kipft(Mol_poss_cs_slab)}kipft)")
Mol_eneg_ms = rd(Mol_eneg - Mol_eneg_cs)
print(f"    1. ESPM_eneg, Mid Strip, Mol_eneg_ms = {Mol_eneg_ms}kNm ({kNm_to_kipft(Mol_eneg_ms)}kipft)")
Mol_ineg_ms = rd(Mol_ineg - Mol_ineg_cs)
print(f"    2. ESPM_ineg, Mid Strip, Mol_ineg_ms = {Mol_ineg_ms}kNm ({kNm_to_kipft(Mol_ineg_ms)}kipft)")
Mol_poss_ms = rd(Mol_poss - Mol_poss_cs)
print(f"    3. ESPM_pos, Mid Strip, Mol_poss_ms = {Mol_poss_ms}kNm ({kNm_to_kipft(Mol_poss_ms)}kipft)")
#SHORT DIRECTION 
print("=== SDIR: Distribution of Mo for End Span with beams between all supports ===")
Mos_eneg = rd(0.16*Mo_SDir)
Mos_ineg = rd(0.70*Mo_SDir)
Mos_poss = rd(0.57*Mo_SDir)
print(f" End Span Panel Moment (negative exterior), Mos_eneg = {Mos_eneg}kNm.")
print(f" End Span Panel Moment (negative interior), Mos_ineg = {Mos_ineg}kNm.")
print(f" End Span Panel Moment (positive), Mos_epos = {Mos_poss}kNm.")
Mos_eneg_cs = rd(0.65*Mos_eneg)
print(f"    1. ESPM_eneg, Col Strip, Mos_eneg_cs = {Mos_eneg_cs}kNm ({kNm_to_kipft(Mos_eneg_cs)}kipft)")
Mos_eneg_cs_beam = rd(0.85*Mos_eneg_cs)
Mos_eneg_cs_slab = rd(0.15*Mos_eneg_cs)
print(f"      1a. Mos_eneg_cs_beam (85%, 8.10.5.7): {Mos_eneg_cs_beam}kNm ({kNm_to_kipft(Mos_eneg_cs_beam)}kipft)")
print(f"      1b. Mos_eneg_cs_slab (15%, 8.10.5.7): {Mos_eneg_cs_slab}kNm ({kNm_to_kipft(Mos_eneg_cs_slab)}kipft)")
Mos_ineg_cs = rd(0.75*Mos_ineg)
print(f"    2. ESPM_ineg, Col Strip, Mos_ineg_cs = {Mos_ineg_cs}kNm ({kNm_to_kipft(Mos_ineg_cs)}kipft)")
Mos_ineg_cs_beam = rd(0.85*Mos_ineg_cs)
Mos_ineg_cs_slab = rd(0.15*Mos_ineg_cs)
print(f"      2a. Mos_ineg_cs_beam (85%, 8.10.5.7): {Mos_ineg_cs_beam}kNm ({kNm_to_kipft(Mos_ineg_cs_beam)}kipft)")
print(f"      2b. Mos_ineg_cs_slab (15%, 8.10.5.7): {Mos_ineg_cs_slab}kNm ({kNm_to_kipft(Mos_ineg_cs_slab)}kipft)")
Mos_poss_cs = rd(0.75*Mos_poss)
print(f"    3. ESPM_pos, Col Strip, Mos_poss_cs = {Mos_poss_cs}kNm ({kNm_to_kipft(Mos_poss_cs)}kipft)")
Mos_poss_cs_beam = rd(0.85*Mos_poss_cs)
Mos_poss_cs_slab = rd(0.15*Mos_poss_cs)
print(f"      3a. Mos_poss_cs_beam (85%, 8.10.5.7): {Mos_poss_cs_beam}kNm ({kNm_to_kipft(Mos_poss_cs_beam)}kipft)")
print(f"      3b. Mos_poss_cs_slab (15%, 8.10.5.7): {Mos_poss_cs_slab}kNm ({kNm_to_kipft(Mos_poss_cs_slab)}kipft)")
Mos_eneg_ms = rd(Mos_eneg - Mos_eneg_cs)
print(f"    1. ESPM_eneg, Mid Strip, Mos_eneg_ms = {Mos_eneg_ms}kNm ({kNm_to_kipft(Mos_eneg_ms)}kipft)")
Mos_ineg_ms = rd(Mos_ineg - Mos_ineg_cs)
print(f"    2. ESPM_ineg, Mid Strip, Mos_ineg_ms = {Mos_ineg_ms}kNm ({kNm_to_kipft(Mos_ineg_ms)}kipft)")
Mos_poss_ms = rd(Mos_poss - Mos_poss_cs)
print(f"    3. ESPM_pos, Mid Strip, Mos_poss_ms = {Mos_poss_ms}kNm ({kNm_to_kipft(Mos_poss_ms)}kipft)")

=== LDIR: Distribution of Mo for End Span with beams between all supports ===
 End Span Panel Moment (negative exterior), Mo_eneg = 95.19kNm.
 End Span Panel Moment (negative interior), Mo_ineg = 416.48kNm.
 End Span Panel Moment (positive), Mo_epos = 339.13kNm.
    1. ESPM_eneg, Col Strip, Mol_eneg_cs = 61.87kNm (45.63kipft)
      1a. Mol_eneg_cs_beam (85%, 8.10.5.7): 52.59kNm (38.79kipft)
      1b. Mol_eneg_cs_slab (15%, 8.10.5.7): 9.28kNm (6.84kipft)
    2. ESPM_ineg, Col Strip, Mol_ineg_cs = 312.36kNm (230.38kipft)
      2a. Mol_ineg_cs_beam (85%, 8.10.5.7): 265.51kNm (195.83kipft)
      2b. Mol_ineg_cs_slab (15%, 8.10.5.7): 46.85kNm (34.55kipft)
    3. ESPM_pos, Col Strip, Mol_poss_cs = 254.35kNm (187.6kipft)
      3a. Mol_poss_cs_beam (85%, 8.10.5.7): 216.2kNm (159.46kipft)
      3b. Mol_poss_cs_slab (15%, 8.10.5.7): 38.15kNm (28.14kipft)
    1. ESPM_eneg, Mid Strip, Mol_eneg_ms = 33.32kNm (24.58kipft)
    2. ESPM_ineg, Mid Strip, Mol_ineg_ms = 104.12kNm (76.79kipft)
    3. ESPM_

In [63]:
print("* * * * * * * * * * * * *")
print("Long Dir. Moment Summary")
Mol_summary = {
    "Beam Moment"   :pd.Series([Mol_ineg_cs_beam,Mol_pos_cs_beam,Mol_eneg_cs_beam,Mol_poss_cs_beam],index=["int_neg","int_pos","ext_neg","ext_pos"]),
    "CS Slab Moment":pd.Series([Mol_ineg_cs_slab,Mol_pos_cs_slab,Mol_eneg_cs_slab,Mol_poss_cs_slab],index=["int_neg","int_pos","ext_neg","ext_pos"]),
    "MS Slab Moment":pd.Series([Mol_ineg_ms,     Mol_pos_ms,     Mol_eneg_ms,     Mol_poss_ms],index=["int_neg","int_pos","ext_neg","ext_pos"])
}
df_Mol_summary = pd.DataFrame(Mol_summary)
print(df_Mol_summary)
print("* * * * * * * * * * * * *")
print("Short Dir. Moment Summary")
Mos_summary = {
    "Beam Moment":pd.Series([Mos_ineg_cs_beam,Mos_pos_cs_beam,Mos_eneg_cs_beam,Mos_poss_cs_beam],index=["int_neg","int_pos","ext_neg","ext_pos"]),
    "CS Slab Moment":pd.Series([Mos_ineg_cs_slab,Mos_pos_cs_slab,Mos_eneg_cs_slab,Mos_poss_cs_slab],index=["int_neg","int_pos","ext_neg","ext_pos"]),
    "MS Slab Moment":pd.Series([Mos_ineg_ms,Mos_pos_ms,Mos_eneg_ms,Mos_poss_ms],index=["int_neg","int_pos","ext_neg","ext_pos"])
}
df_Mos_summary = pd.DataFrame(Mos_summary)
print(df_Mos_summary)
print("* * * * * * * * * * * * *")


* * * * * * * * * * * * *
Long Dir. Moment Summary
         Beam Moment  CS Slab Moment  MS Slab Moment
int_neg       265.51           46.85          104.12
int_pos       132.75           23.43           52.06
ext_neg        52.59            9.28           33.32
ext_pos       216.20           38.15           84.78
* * * * * * * * * * * * *
Short Dir. Moment Summary
         Beam Moment  CS Slab Moment  MS Slab Moment
int_neg       209.55           36.98           82.18
int_pos       104.77           18.49           41.09
ext_neg        41.51            7.32           26.30
ext_pos       170.64           30.11           66.91
* * * * * * * * * * * * *


In [64]:
#Strip / width
bw_sup = width_cs #mm
bw_mid = width_ms #mm
print(f"bw_mid = {bw_mid}mm | bw_supp = {width_cs}mm")

#LONG DIR: RSB SPACING
#Moment for bottom reinf
LDir_mid_Mo_bot = Mol_poss_ms
LDir_sup_Mo_bot = Mol_poss_cs_slab
#moment for top reinf
LDir_sup_Mo_top = Mol_ineg_cs_slab

#SHRT DIR: RSB SPACING
#Moment for bottom reinf
SDir_mid_Mo_bot = Mos_poss_ms
SDir_sup_Mo_bot = Mos_poss_cs_slab
#Moment for top reinf
SDir_sup_Mo_top = Mos_ineg_cs_slab




bw_mid = 4395.0mm | bw_supp = 2865.0mm


In [65]:
#Material Properties
fc = 27       #concrete compressive strength, MPa
fy = 230        #steel yield strength, MPa
fyt = 230       #steel yield strength of ties, MPa
Es = 200000     #Steel Elasticity, GPa
print(f"==== MATERIAL PROPERTIES ====\nfc = {fc}MPa, fy = {fy}MPa, fyt = {fyt}MPa")

#Tensile Reinforcement Detail
d_main = 12     #diameter of main bars, mm
d_tie = 12      #diameter of tie bars, mm
n_tieleg = 2    #no. of tie legs
sc = 25         #steel cover

def MomentCap(bw,h,layer, n1, n2, Mu):
    if layer > 1:
        As1 = n1*pi*0.25*d_main**2
        As2 = n2*pi*0.25*d_main**2
        As = rd(As1 + As2)
        y = (As1*0 + As2*(d_main + 25))/As
        dt = h - sc - d_tie - d_main/2      #reinforcement farthest from the compression face
        d = rd(dt - y)                          #centroid of steel
        print(f"1. dmain = {d_main} | n1 = {n1}pcs, n2 = {n2}pcs | d = {d}mm")
    else:
        dt = h - sc - d_tie - d_main/2      #reinforcement farthest from the compression face
        d = dt                              #centroid of steel
        As = rd( pi*d_main**2*0.25*n1 )
        print(f"1. dmain = {d_main} | n = {n1}pcs | dt = {dt}mm")
    
    s2 = round((bw-sc*2-d_tie*2)/(n1-1))        #spacing of the long reinf., mm
    print(f"2. As = {As}mm2 | spacing (mainbar) = {s2}mm.")
    #Beta Factor
    if fc <= 28:
        beta1 = 0.85
    elif 28 < fc < 55:
        beta1 = round(0.85 - 0.05*(fc-28)/7,4)
    else:
        beta1 = 0.65
    #Equivalent compression block
    a = As*fy/(0.85*fc*bw)
    c = a/beta1
    #Rho for Balanced Strain Condition
    eu = 0.003      #crushing strain of concrete
    ety = 0.002      #ACI 21.2.2.1, net tensile strain value in the extreme layer of long ten reinf. if compression controlled
    ey = fy/Es     
    rho_b = round(0.85*beta1*fc*eu/(fy*(eu+ety)),5)
    #Rho for Actual Strain Condition
    et = eu*(dt-c)/c    #net tensile strain in extreme layer of long ten reinf.
    rho_act = round(0.85*beta1*fc*dt*eu/(fy*d*(eu+et)),5)
    #Rho for maximum reinforcement ratio for a tension-controlled beam
    rho_ten = round(0.85*beta1*fc*eu/(fy*(eu+0.005)),5)
    print(f"3. rho_bal: {rho_b} | rho_act: {rho_act} | rho_ten: {rho_ten}")
    def phi():
        if et <= ety:
            phi = 0.65
        elif ety < et < 0.005:
            phi = 0.65 + 0.25*(et - ey)/(0.005 - ey)
        else:
            phi = 0.90
        return phi
    print(f"4. phi_flex = {phi()} | beta1 = {beta1}")
    #Bending Capacity
    Mn = rd(0.85*fc*a*bw*(d-a/2)*(0.001**2))      #Bending Capacity, kN-m
    phiMn = rd(phi()*Mn) #kN-m
    if phiMn > Mu:
        print(f"5. phiMn = {phiMn}kN-m > Mu = {Mu}kN-m. SAFE!")
    else:
        print(f"5. phiMn = {phiMn}kN-m < Mu = {Mu}kN-m. UNSAFE!")
        exit("***** PROGRAM TERMINATED *****")
    return Mn, phiMn, d, s2, n1, n2, layer

==== MATERIAL PROPERTIES ====
fc = 27MPa, fy = 230MPa, fyt = 230MPa


In [67]:
#MomentCapacity
print("=== LONG DIR: MIDSPAN BOT BARS ===")
lmid_bot = MomentCap(bw_mid,S1.thk,1,40,0,LDir_mid_Mo_bot)
print("=== LONG DIR: SUPSPAN BOT BARS ===")
lsup_bot = MomentCap(bw_sup,S1.thk,1,28,0,LDir_sup_Mo_bot)
print("=== LONG DIR: SUPSPAN TOP BARS ===")
lsup_bot = MomentCap(bw_sup,S1.thk,1,28,0,LDir_sup_Mo_top)

print("=== SHRT DIR: MIDSPAN BOT BARS ===")
smid_bot = MomentCap(bw_mid,S1.thk,1,40,0,SDir_mid_Mo_bot)
print("=== SHRT DIR: SUPSPAN BOT BARS ===")
ssup_bot = MomentCap(bw_sup,S1.thk,1,28,0,SDir_sup_Mo_bot)
print("=== SHRT DIR: SUPSPAN TOP BARS ===")
ssup_bot = MomentCap(bw_sup,S1.thk,1,28,0,SDir_sup_Mo_top)


=== LONG DIR: MIDSPAN BOT BARS ===
1. dmain = 12 | n = 40pcs | dt = 107.0mm
2. As = 4523.89mm2 | spacing (mainbar) = 111mm.
3. rho_bal: 0.05089 | rho_act: 0.00962 | rho_ten: 0.03181
4. phi_flex = 0.9 | beta1 = 0.85
5. phiMn = 95.37kN-m > Mu = 84.78kN-m. SAFE!
=== LONG DIR: SUPSPAN BOT BARS ===
1. dmain = 12 | n = 28pcs | dt = 107.0mm
2. As = 3166.73mm2 | spacing (mainbar) = 103mm.
3. rho_bal: 0.05089 | rho_act: 0.01033 | rho_ten: 0.03181
4. phi_flex = 0.9 | beta1 = 0.85
5. phiMn = 66.51kN-m > Mu = 38.15kN-m. SAFE!
=== LONG DIR: SUPSPAN TOP BARS ===
1. dmain = 12 | n = 28pcs | dt = 107.0mm
2. As = 3166.73mm2 | spacing (mainbar) = 103mm.
3. rho_bal: 0.05089 | rho_act: 0.01033 | rho_ten: 0.03181
4. phi_flex = 0.9 | beta1 = 0.85
5. phiMn = 66.51kN-m > Mu = 46.85kN-m. SAFE!
=== SHRT DIR: MIDSPAN BOT BARS ===
1. dmain = 12 | n = 40pcs | dt = 107.0mm
2. As = 4523.89mm2 | spacing (mainbar) = 111mm.
3. rho_bal: 0.05089 | rho_act: 0.00962 | rho_ten: 0.03181
4. phi_flex = 0.9 | beta1 = 0.85
5. ph